<a href="https://colab.research.google.com/github/rebeccaxchu/Real-Property-Assessments/blob/main/scrips%20/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install azure-storage-blob
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00


In [2]:
#import libraries
import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [3]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# specify the path to your JSON configuration file
config_file_path= '/content/gdrive/MyDrive/Colab Notebooks/mdproperty/credentials.json'

# load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

Connection_String_AZURE_STORAGE = config["ConnectionString"]
CONTAINER_AZURE= 'mdrealproperty'



In [5]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [6]:
#initialize the blobserviceclient
blob_service_client = BlobServiceClient.from_connection_string(Connection_String_AZURE_STORAGE)

#get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

md_property_df = pd.DataFrame()

In [7]:
#list all blobs in specified container
blob_list = container_client.list_blobs()

for blob in blob_list:
  print(blob.name)
  blob_client = container_client.get_blob_client(blob=blob.name)
  blob_data = blob_client.download_blob()
  blob_content = blob_data.readall().decode('utf-8')
  df = pd.read_csv(StringIO(blob_content))

property.csv


<ipython-input-7-81df1378b9d7>:9: DtypeWarning: Columns (21,47,89,124,166,173,178,186,199,201,203,206,207,208) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


In [10]:
  # Display the head of the DataFrame
  df.shape



(35228, 209)

In [12]:
 # since I have only one csv, I am doing to do the following instructions
md_property_df = df.copy()
